### Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetWithBN(nn.Module):
    def __init__(self):
        super(UNetWithBN, self).__init__()
        
        # Encoder Block 1
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 12, 3, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(inplace=True),
        )
        
        # Encoder Block 2
        self.enc2 = nn.Sequential(
            nn.Conv2d(12, 24, 3, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(inplace=True),
        )
        
        # Encoder Block 3
        self.enc3 = nn.Sequential(
            nn.Conv2d(24, 34, 3, padding=1),
            nn.BatchNorm2d(34),
            nn.ReLU(inplace=True),
        )
        
        # Decoder Block 3
        self.dec3 = nn.Sequential(
            nn.Conv2d(34, 24, 3, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(inplace=True),
        )
        
        # Decoder Block 2
        self.dec2 = nn.Sequential(
            nn.Conv2d(24, 12, 3, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(inplace=True),
        )
        
        # Final layer
        self.final = nn.Conv2d(12, 1, 1)
        
        # Pooling and Upsampling
        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        p1 = self.pool(e1)
        
        e2 = self.enc2(p1)
        p2 = self.pool(e2)
        
        e3 = self.enc3(p2)
        
        # Decoder
        d3 = self.dec3(self.up(e3))
        d2 = self.dec2(self.up(d3))
        
        out = self.final(d2)
        return out

In [ ]:
from torchvision import transforms

def predict(model, img):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    
    # Load and preprocess image
    image = img.convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output) > 0.9
    
    return prediction.squeeze().cpu().numpy()

In [ ]:
model_path= 'best_model.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNetWithBN().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

### Run

In [ ]:
import pyautogui

In [ ]:
# use this to find the coordinate
pyautogui.position()


In [ ]:

pyautogui.screenshot(region=(20,210, 950, 500))

### Auto streeing

In [ ]:
import cv2
import numpy as np
import pyautogui
import time

def get_steering_angle(masked_image):
    # Get image dimensions
    height, width = masked_image.shape
    
    # Define the region of interest (bottom half of the image)
    roi_height = height // 2
    roi = masked_image[roi_height:height, :]
    
    # Find contours in the ROI
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Find the largest contour (assuming it's the road)
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Calculate the center of the contour
        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
        else:
            cx = width // 2
            
        # Calculate the offset from the center
        center_offset = cx - (width // 2)
        
        # Convert offset to steering angle
        # You may need to adjust these values based on your specific needs
        max_angle = 45
        steering_angle = (center_offset / (width // 2)) * max_angle
        
        return steering_angle
    
    return 0

def control_vehicle(steering_angle):
    # Define steering sensitivity
    steering_sensitivity = 0.1
    
    # Apply steering control using PyAutoGUI
    if abs(steering_angle) > 5:  # Add dead zone to prevent small oscillations
        if steering_angle < 0:
            # Turn left
            pyautogui.keyDown('left')
            time.sleep(0.1)
            print("left")
            # time.sleep(abs(steering_angle) * steering_sensitivity)
            # print('left :',abs(steering_angle) * steering_sensitivity)
            pyautogui.keyUp('left')
        else:
            # Turn right
            pyautogui.keyDown('right')
            time.sleep(0.1)
            print("right")
            # time.sleep(abs(steering_angle) * steering_sensitivity)
            # print('right :',abs(steering_angle) * steering_sensitivity)
            pyautogui.keyUp('right')
    
    # pyautogui.keyDown('up')
    # time.sleep(0.1)
    # pyautogui.keyUp('up')

def main(runfor):
    # Add safety delay to switch to your game window
    print("Switching to game window in 5 seconds...")
    time.sleep(5)
    pyautogui.click(x=100, y=400)
    
    start_time = time.time() 
    runfor = runfor
    while (time.time() - start_time) < runfor:
        try:
            pyautogui.click(x=100, y=400)
            
            im = pyautogui.screenshot(region=(20,210, 950, 500))
            
            masked_road =  predict(model, im)

            angle = get_steering_angle(masked_road.astype(np.uint8))
            
            control_vehicle(angle)
                      
        except Exception as e:
            print(f"Error: {e}")
            break
    


In [ ]:
main(60)